In [1]:
import pandas as pd
import numpy as np
import datetime as dt

**Read Datasets**

In [3]:
df_movies = pd.read_csv("archive/movies_metadata.csv")
df_ratings = pd.read_csv("archive/ratings_small.csv")

In [4]:
df_ratings.shape

(100004, 4)

In [36]:
#reduce number of users
#df_ratings = df_ratings[df_ratings.userId <= 50]

In [5]:
df_movies.shape

(45466, 24)

In [6]:
df_movies.columns,df_movies.dtypes

(Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
        'imdb_id', 'original_language', 'original_title', 'overview',
        'popularity', 'poster_path', 'production_companies',
        'production_countries', 'release_date', 'revenue', 'runtime',
        'spoken_languages', 'status', 'tagline', 'title', 'video',
        'vote_average', 'vote_count'],
       dtype='object'),
 adult                     object
 belongs_to_collection     object
 budget                    object
 genres                    object
 homepage                  object
 id                        object
 imdb_id                   object
 original_language         object
 original_title            object
 overview                  object
 popularity                object
 poster_path               object
 production_companies      object
 production_countries      object
 release_date              object
 revenue                  float64
 runtime                  float64
 spoken_lan

In [7]:
#From OFAiR paper
# delete this row
df_movies.drop(35587, inplace=True)

# casting it from str to float
df_movies['popularity'] = df_movies['popularity'].astype(float)

# calculate mean pop for filling in the missing values
mean_pop = (df_movies.loc[df_movies['popularity'].isnull() == False, 'popularity']).mean()

# filling the missing value with mean
df_movies['popularity'].fillna(mean_pop, inplace=True)

# how many missing values?
print('missing?', df_movies['popularity'].isnull().sum())

print('# above mean', len(df_movies[df_movies['popularity'] >= mean_pop]))
print('# below mean', len(df_movies[df_movies['popularity'] < mean_pop]))

missing? 0
# above mean 13076
# below mean 32389


In [8]:
df_movies['revenue'] = df_movies['revenue'].astype(float)
mean_rev = (df_movies.loc[df_movies['revenue'].isnull() == False, 'revenue']).mean()
df_movies['revenue'].fillna(mean_rev, inplace=True)
print('missing?', df_movies['revenue'].isnull().sum())

print('# above mean', len(df_movies[df_movies['revenue'] >= mean_rev]))
print('# below mean', len(df_movies[df_movies['revenue'] < mean_rev]))

missing? 0
# above mean 4215
# below mean 41250


In [9]:
df_movies['runtime'] = df_movies['runtime'].astype(float)
mean_rt = (df_movies.loc[df_movies['runtime'].isnull() == False, 'runtime']).mean()
median_rt =(df_movies.loc[df_movies['runtime'].isnull() == False, 'runtime']).median()

print (mean_rt)
print (median_rt)

df_movies['runtime'].fillna(mean_rt, inplace=True)
print('missing?', df_movies['runtime'].isnull().sum())

print('# above mean', len(df_movies[df_movies['runtime'] >= mean_rt]))
print('# below mean', len(df_movies[df_movies['runtime'] < mean_rt]))

94.12819945578833
95.0
missing? 0
# above mean 23047
# below mean 22418


In [10]:
print('missing?', df_movies['release_date'].isnull().sum())
df_movies.drop(df_movies[df_movies['release_date'].isnull()==True].index.tolist(), inplace=True)
df_movies.drop(df_movies[df_movies['release_date'] == '1'].index.tolist(), inplace=True)
df_movies.drop(df_movies[df_movies['release_date'] == '12'].index.tolist(), inplace=True)
# deleting them!

print('missing?', df_movies['release_date'].isnull().sum())

missing? 87
missing? 0


In [11]:
df_movies.columns.values, df_movies.shape

(array(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
        'id', 'imdb_id', 'original_language', 'original_title', 'overview',
        'popularity', 'poster_path', 'production_companies',
        'production_countries', 'release_date', 'revenue', 'runtime',
        'spoken_languages', 'status', 'tagline', 'title', 'video',
        'vote_average', 'vote_count'], dtype=object),
 (45376, 24))

In [12]:
# original cleaning logic
# #Clean the data
# df_ratings = df_ratings.drop("timestamp",axis=1)
# df_ratings["rating"] = df_ratings["rating"].astype("int")
# df_movies = df_movies[df_movies["id"].str.isnumeric()]
df_movies["id"] = df_movies["id"].astype("int")

# #How should we populate missing dates???
# df_movies.release_date = df_movies.release_date.fillna('1989-12-31')
# df_movies = df_movies.drop(df_movies[pd.to_datetime(df_movies.release_date,errors="coerce").isna()].index)
df_movies.release_date = pd.to_datetime(df_movies.release_date)

In [13]:
#Merge the two datasets using an inner to prevent orphaned ids??
df_movies = df_movies.merge(df_ratings,left_on="id",right_on="movieId",how="inner")

In [14]:
df_movies.shape

(44965, 28)

In [15]:
#Only pick movies with 50 rating or over
df_rating_count = df_movies.groupby("userId")["movieId"].count()
df_rating_count[df_rating_count >= 50].index
df_movies = df_movies[df_movies.userId.isin(df_rating_count[df_rating_count >= 50].index)] 

In [16]:
df_ratings = df_movies[["userId","movieId","rating"]].sort_values(by="userId")

In [17]:
#Create the ratings.csv for librec-auto
df_ratings.to_csv("ratings.csv",index=None,header=False)

In [18]:
#creating the protected feature bins
avg_vote = df_movies.vote_count.mean()
avg_rev = df_movies.revenue.mean()
avg_runtime = df_movies.runtime.mean()
cutoff_date = dt.datetime.strptime("1990","%Y")

In [19]:
#create dummies for each protected feature
df_pop = pd.get_dummies(df_movies.vote_count.transform( lambda x: "pop" if x >= avg_vote else "notpop"))
df_date =  pd.get_dummies(df_movies.release_date.transform( lambda x: "old" if x >= cutoff_date else "new"))
df_rev =  pd.get_dummies(df_movies.revenue.transform( lambda x: "high" if x >= avg_rev else "low"))
df_len =  pd.get_dummies(df_movies.runtime.transform( lambda x: "long" if x >= avg_runtime else "short"))

In [20]:
df_movies.id, df_date

(0          949
 1          949
 2          949
 3          949
 5          949
          ...  
 44959     3104
 44960    64197
 44961    64197
 44962    64197
 44964    49280
 Name: id, Length: 34827, dtype: int64,
        new  old
 0        0    1
 1        0    1
 2        0    1
 3        0    1
 5        0    1
 ...    ...  ...
 44959    1    0
 44960    0    1
 44961    0    1
 44962    0    1
 44964    1    0
 
 [34827 rows x 2 columns])

In [21]:
#create full item feature table
df_item_features = pd.concat([df_movies["id"],df_pop,df_date,df_rev,df_len], axis=1)
df_item_features = df_item_features.set_index("id").stack().reset_index()
df_item_features.columns = ["item_id","feature","value"]
df_item_features.shape 

(278616, 3)

In [22]:
df_item_features = df_item_features[df_item_features.value == 1]
df_item_features = df_item_features.drop_duplicates()
df_item_features.shape

(11024, 3)

In [55]:
#Remove duplicates
# df_item_features = df_item_features[~df_item_features.duplicated()]
# df_item_features[~df_item_features.duplicated()].shape,df_item_features[df_item_features.duplicated()].shape

In [23]:
#Single out specifc feature
df_item_features = df_item_features[(df_item_features["feature"]=='old') | (df_item_features["feature"]=='new') ]


In [24]:
df_item_features.to_csv("item-features.csv",header=False,index=None)